In [1]:
# prompt: write code import langchain

# !pip install langchain

# from langchain.llms import OpenAI
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain


## simple sequential chain

In [7]:
from re import template
from langchain_core.output_parsers import StrOutputParser
from dotenv import  load_dotenv
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate


In [8]:

load_dotenv()

True

In [16]:

model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
print("Model initialized successfully.")


parser = StrOutputParser()

prompt = PromptTemplate(
    template="give me detailed report about {name}?",
    input_variables=["name"],
)

chain = prompt | model | parser


Model initialized successfully.


In [15]:
chain.invoke({"name": "sharyar"})

'Betrayal, tyranny, stories, transformation, ambiguity.'

## here is bit complex chain

In [11]:
prompt1 = PromptTemplate(
    template="extract five word summary from {topic}?",
    input_variables=["topic"],
)

chain = prompt | model | parser | prompt1 | model | parser

In [12]:
chain.invoke({"name": "love"})

'Love: Complex, biological, psychological, social.'

In [13]:
# chain.get_graph().print_ascii()

##**form a simple paralel chain**

In [ ]:
prompt2 = PromptTemplate(
    template="write quiz from given paragraph {topic}?",
    input_variables=["topic"],
)

In [ ]:
prompt3 = PromptTemplate(
    template="write notes from given paragraph {topic}?",
    input_variables=["topic"],
)

In [ ]:
prompt4 = PromptTemplate(
    template="merge the provided notes into single document and quiz notes--> {notes} quiz--> {quiz}?",
    input_variables=["notes",'quiz'],
)

In [ ]:
from langchain_core.runnables import RunnableParallel
parallel_chain = RunnableParallel(
    {
      'notes' : prompt2 | model | parser,
     'quiz': prompt3 | model | parser
     }
)

In [ ]:
merge_chain = prompt4 | model | parser

In [ ]:
chain = parallel_chain | merge_chain

In [ ]:
chain.invoke({"topic": "A simple life is one that embraces minimalism, contentment, and mindfulness. It’s about finding joy in the little things—like a quiet morning, meaningful conversations, or a walk in nature—rather than constantly chasing material success or external validation. By reducing unnecessary stress and distractions, a simple life creates space for clarity, peace, and genuine connection with yourself and others. It’s not about having less, but about making room for what truly matter"})

"## A Simple Life: Understanding and Embracing It\n\nThis document combines key ideas about living a simple life with a quiz to test your understanding.\n\n**Simple Life: Key Ideas**\n\n*   **Core Principles:**\n    *   Minimalism: Reducing unnecessary possessions and commitments.\n    *   Contentment: Finding satisfaction with what you have, rather than constantly striving for more.\n    *   Mindfulness: Paying attention to the present moment without judgment.\n\n*   **Focus:**\n    *   Joy in small things: Appreciating the beauty and pleasure found in everyday experiences like quiet mornings, meaningful conversations, and nature walks.\n    *   Internal validation, not external: Seeking approval and satisfaction from within, rather than relying on the opinions of others.\n\n*   **Benefits:**\n    *   Reduced stress/distractions: Eliminating unnecessary clutter and commitments to create a calmer and more focused life.\n    *   Clarity, peace: Gaining a clearer understanding of your va

## here are conditional chain

In [27]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# Define the structured output model
class SentimentOutput(BaseModel):
    sentiment: str = Field(..., description="The sentiment of the feedback, either 'positive' or 'negative'.")

# Create a PydanticOutputParser
pydantic_parser = PydanticOutputParser(pydantic_object=SentimentOutput)


In [30]:
cn_prompt = PromptTemplate(
    template="give feedback in negative or positive  sentiment {topic}? ,{format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
)

In [44]:
classifier_chain = cn_prompt | model | pydantic_parser

In [45]:
classifier_chain.invoke({"topic" : "this is a terible product"}).sentiment

'negative'

In [46]:
prompt_negative = PromptTemplate(
    template="write a response to this feedback {topic}?",
    input_variables=["topic"]
)
prompt_positive = PromptTemplate(
    template="write a response to this feedback {topic}?",
    input_variables=["topic"]
)

In [47]:
from langchain_core.runnables import RunnableBranch
from langchain_core.runnables import RunnableLambda

In [48]:

branch_chain = RunnableBranch(
    (lambda x: x.sentiment == "positive", prompt_positive | model | parser),
    (lambda x: x.sentiment == "negative", prompt_negative | model | parser),
    RunnableLambda(lambda x: "could not determine sentiment")
)


In [49]:
chain = classifier_chain | branch_chain

In [50]:
chain.invoke({"topic" : "this is a terible product"})

'Okay, I understand the feedback is negative. Here\'s a breakdown of how I can respond, depending on the context and what you want to achieve:\n\n**General Principles for Responding to Negative Feedback:**\n\n*   **Acknowledge and Validate:** Show that you\'ve heard and understood the feedback.\n*   **Be Empathetic:** Try to see things from the other person\'s perspective.\n*   **Apologize (If Appropriate):** If you made a mistake or caused inconvenience, a sincere apology is crucial.\n*   **Explain (If Necessary):** Provide context or reasoning, but avoid making excuses.\n*   **Offer a Solution or Plan:** If possible, suggest how you\'ll address the issue or prevent it from happening again.\n*   **Thank the Person:** Express gratitude for the feedback, as it helps you improve.\n*   **Keep it Professional:** Maintain a respectful and courteous tone.\n\n**Possible Responses (Choose the one that best fits your situation):**\n\n**1. Simple Acknowledgment and Gratitude:**\n\n*   "Thank you